Installing the required packages and importing required libraries:

In [1]:
!pip install neuspell
!pip install -e neuspell/
!git clone https://github.com/neuspell/neuspell; cd neuspell
!pip install DeepTextSearch

import os
os.chdir("/content/neuspell")

!pip install -r /content/neuspell/extras-requirements.txt
!python -m spacy download en_core_web_sm

#Unzipping the multi-linguistic packages
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip *.zip

  Using cached neuspell-1.0.0-py3-none-any.whl (184 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\scripts\\__init__.py'
Consider using the `--user` option or check the permissions.

ERROR: neuspell/ is not a valid editable requirement. It should either be a path to a local project or a VCS URL (beginning with bzr+http, bzr+https, bzr+ssh, bzr+sftp, bzr+ftp, bzr+lp, bzr+file, git+http, git+https, git+ssh, git+git, git+file, hg+file, hg+http, hg+https, hg+ssh, hg+static-http, svn+ssh, svn+http, svn+https, svn+svn, svn+file).
'git' is not recognized as an internal or external command,
operable program or batch file.


  Created wheel for DeepTextSearch: filename=DeepTextSearch-0.3-py3-none-any.whl size=5960 sha256=b36ab999be0c749ef2766a9b5b17aafab19a9ebf0a68595d245b990aeb6dbb97
  Stored in directory: c:\users\anoosh\appdata\local\pip\cache\wheels\05\48\fd\009043091e6f3b917d1beb9bdfc0711a400f50c8bc3fbcf8fd
Successfully built DeepTextSearch


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/neuspell'

In [3]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
from DeepTextSearch import TextEmbedder,TextSearch, LoadData
from nltk.corpus import wordnet
import pandas as pd
from neuspell import BertChecker

data folder is set to `/content/neuspell/neuspell/../data` script


#**Contextual Spell-Check**

In [5]:
spellcheck = BertChecker()
spellcheck.from_pretrained(
    ckpt_path=f"/content/multi_cased_L-12_H-768_A-12"  # "<folder where the model is saved>"
)

/content/neuspell/neuspell/../data/checkpoints/subwordbert-probwordnoise created
Pretrained model downloading start (may take few seconds to couple of minutes based on download speed) ...
Pretrained model download success
loading vocab from path:/content/neuspell/neuspell/../data/checkpoints/subwordbert-probwordnoise/vocab.pkl
initializing model


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Number of parameters in the model: 185211810
Loading model params from checkpoint dir: /content/neuspell/neuspell/../data/checkpoints/subwordbert-probwordnoise


#**Recommendation System**

###**English**

Applying spell-checker:

In [6]:
X=input("Enter Product Name:")
y=spellcheck.correct(X)
print(y)

Enter Product Name:kangaro pockt



kangaroo pockets


POS Tagging:

In [7]:
#We are using POS tagging to select relevent words from the given query.
def preprocess(sent):
    sent = nltk.word_tokenize(sent)
    sent = nltk.pos_tag(sent)
    return sent

In [8]:
sent = preprocess(y)
l=[]
for i in sent:
  if i[1]=='NNS' or i[1]=='NN':
    l.append(i[0])
print(l)

['kangaroo', 'pockets']


Query expansion using WordNet:

In [9]:
#We use query expansion to get synonyms of words, so that we can get more relevent recommendations.
query=""
for i in l:
  query+=i
  synset = wordnet.synsets(i)
  query+=" "+synset[0].lemmas()[0].name()+" "
print(query)

kangaroo kangaroo pockets pocket 


Creating dictionary between product name & description:

In [14]:
df=pd.read_csv("walmart_com-ecommerce_product_details__20190311_20191001_sample.csv")
df.head()

,Uniq Id,Crawl Timestamp,Product Url,Product Name,Description,List Price,Sale Price,Brand,Item Number,Gtin,Package Size,Category,Postal Code,Available
0,f707c5776d0818126a36b3d3f14d7f6a,2019-03-17 07:42:16 +0000,https://www.walmart.com/ip/Compatible-1988-199...,Compatible 1988 - 1993 Pontiac LeMans Fog Ligh...,High-Quality TYC/DEPO-brand item; with Reflect...,45.95,45.95,Go-Parts,NaN,799193152947,NaN,Auto & Tires|Automotive Replacement Parts|Car ...,NaN,False
1,95ed787e89bd1c2b536a8585510a2f73,2019-03-17 08:29:29 +0000,https://www.walmart.com/ip/14k-White-Gold-Diam...,14k White Gold Diamond Double Heart Pendant - ...,14k White Gold Diamond Double Heart Pendant - ...,154.20,154.20,JewelryWeb,NaN,190772364513,NaN,Jewelry & Watches|Womens Jewelry & Watches|Wom...,NaN,False
2,40ede783dd78605f2c563b3ee60a7a93,2019-03-17 07:02:01 +0000,https://www.walmart.com/ip/Multi-Surface-Paint...,Adhesive Blue Painter Masking Tape 5.6 Mil Thi...,Masking Tape: Masking tape is low adhesive tap...,234.95,199.71,PackagingSuppliesByMail,NaN,640791636141,NaN,Home Improvement|Paint|Paint Tools and Supplie...,NaN,True
3,b5f44e518b12cc77fe19e6091a487101,2019-03-17 07:34:40 +0000,https://www.walmart.com/ip/Cycle-Pirates-Chrom...,Cycle Pirates Chrome Cast Switch Housing Left ...,Add that extra little bling to get your bike m...,59.95,59.95,Cycle Pirates,NaN,633835779256,NaN,NaN,NaN,True
4,a59dd2d01e705d12151c551582895717,2019-03-17 07:37:29 +0000,https://www.walmart.com/ip/Suspension-Control-...,Suspension Control Arm Rear Lower MOOG RK642963,Suspension Control Arm Rear Lower MOOG RK64296...,110.18,110.18,MOOG,NaN,80066087139,NaN,Auto & Tires|Automotive Replacement Parts|Repl...,NaN,True


In [15]:
df1=df.set_index("Description", inplace = False)


In [16]:
df2=df1.to_dict()
dict1=df2['Product Name']

Loading the data and embedding the data in a pickle file:

In [23]:
data = LoadData().from_csv("walmart_com-ecommerce_product_details__20190311_20191001_sample.csv")
TextEmbedder().embed(corpus_list=data)
corpus_embedding = TextEmbedder().load_embedding()

Input the text Column Name Please ? : Description
Embedding data allready present, Do you want Embed & Save Again? Enter yes or no
yes



Embedding data Saved Successfully Again!
['corpus_list_data.pickle', 'corpus_embeddings_data.pickle']
Embedding data Loaded Successfully!
['corpus_list_data.pickle', 'corpus_embeddings_data.pickle']


Searching the 10 most relevant products based on the query:

In [24]:
n=10
t=TextSearch().find_similar(query_text=query,top_n=n)
for i in range(n):
    t[i]['text']=dict1[t[i]['text']]
    print(t[i])

{'index': 25, 'text': "ProSphere Boys' Daemen College Gameday Pullover Hoodie", 'score': 0.247256}
{'index': 24, 'text': "ProSphere Boys' Daemen College Gameday Pullover Hoodie", 'score': 0.247256}
{'index': 23, 'text': "ProSphere Boys' Daemen College Gameday Pullover Hoodie", 'score': 0.247256}
{'index': 14, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.14584301}
{'index': 17, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.14584301}
{'index': 16, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.14584301}
{'index': 15, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.14584301}
{'index': 20, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.14584301}
{'index': 13, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.14584301}
{'index': 12, 'text': "Men's Big & Tall Harbor Bay Space-Dye Piqué Polo Shirt", 'score': 0.145843

###**Other Languages**

#####**Arabic**

In [ ]:
X1=input("Search Engine:")
y1=spellcheck.correct(X1)
print(y1)

Search Engine:أجيب في الثاني.
أجيب في الثاني .


In [ ]:
data1 = LoadData().from_text("/wiki_books_test_1.txt")
TextEmbedder().embed(corpus_list=data1)
corpus_embedding = TextEmbedder().load_embedding()

Embedding data allready present, Do you want Embed & Save Again? Enter yes or no
yes



Embedding data Saved Successfully Again!
['corpus_embeddings_data.pickle', 'corpus_list_data.pickle']
Embedding data Loaded Successfully!
['corpus_embeddings_data.pickle', 'corpus_list_data.pickle']


In [ ]:
TextSearch().find_similar(query_text=y1,top_n=10)

[{'index': 383, 'score': 0.79338586, 'text': 'والثاني: تجب فيهما.'},
 {'index': 273, 'score': 0.74463975, 'text': 'والثاني: بالثاني.'},
 {'index': 433, 'score': 0.7429859, 'text': 'والثاني: على القولين.'},
 {'index': 32,
  'score': 0.7291739,
  'text': 'قلت: الثاني أصح، وصححه الاكثرون والله أعلم.'},
 {'index': 242, 'score': 0.7224536, 'text': 'وعلى الثاني: بعذر.'},
 {'index': 460, 'score': 0.71352804, 'text': 'والثاني: استحبابه.'},
 {'index': 446, 'score': 0.71352804, 'text': 'والثاني: استحبابه.'},
 {'index': 8885, 'score': 0.7128246, 'text': 'ومن وجه بالقسم الثاني .'},
 {'index': 205, 'score': 0.68822443, 'text': 'والثاني: لا.'},
 {'index': 4641, 'score': 0.68177044, 'text': 'ولواحق خبر ثالث.'}]

#####**Hindi**

In [ ]:
X_hindi=input("Search Engine:")
y_hindi=spellcheck.correct(X_hindi)
print(y_hindi)

Search Engine:निर्देशक
निर्देशक


In [ ]:
data_hindi = LoadData().from_csv("hindi.csv")
TextEmbedder().embed(corpus_list=data_hindi)
corpus_embedding = TextEmbedder().load_embedding()

Input the text Column Name Please ? : text
Embedding data allready present, Do you want Embed & Save Again? Enter yes or no
yes



Embedding data Saved Successfully Again!
['corpus_list_data.pickle', 'corpus_embeddings_data.pickle']
Embedding data Loaded Successfully!
['corpus_list_data.pickle', 'corpus_embeddings_data.pickle']


In [ ]:
TextSearch().find_similar(query_text=y_hindi,top_n=10)

[{'index': 410,
  'score': 0.67329043,
  'text': 'बैनर :\nयूटीवी मोशन पिक्चर्स\n\nनिर्माता :\nरॉनी स्क्रूवाला\n\nनिर्देशन एवं संगीत :\nविशाल भारद्वाज\n\nगीत :\nगुलजार\n\nकलाकार :\nशाहिद कपूर, प्रियंका चोपड़ा, अमोल गुप्ते, तेनजिंग नीमा, चंदन रॉय सान्याल और शिव सुब्रमण्यम\n\n* केवल वयस्कों के लिए\n\n\n\nफरहा खान (ओम शांति ओम), अनुराग कश्यप (देव डी) और विजय कृष्णा आचार्य (टशन) के बाद विशाल भारद्वाज ने भी सत्तर और अस्सी के दशक की कहानी को नए प्रस्तुतिकरण के साथ पेश किया है। ‘मकबूल’ और ‘ओंकारा’ जैसी फिल्म बनाने वाले इस निर्देशक ने हमशक्ल भाइयों और अपराध पर बन चुकी कई फिल्मों के आधार पर ‘कमीने’ बनाई है।\n\nगुड्डू (शाहिद कपूर) सीधा-सादा है। हकलाता है। चार्ली (शाहिद कपूर) तुतलाता है। ‘स’ को ‘फ’ बोलता है। वह अमीर बनने के लिए हर रास्ते पर चलने के लिए तैयार है। उसका मानना है कि जीवन में दो रास्ते होते हैं। ‘शॉर्टकट’ और ‘छोटा शॉर्टकट’। दोनों भाई एक-दूसरे से नफरत करते हैं। तीन साल से दोनों ने एक-दूसरे की शक्ल नहीं देखी है।\n\nस्वीटी (प्रियंका चोपड़ा) गुड्डू को चाहती है और उसका भाई भोपे (अमोल गुप्ते

#####**Japanese**

In [ ]:
X_japanese=input("Search Engine:")
# y_japanese=spellcheck.correct(X_japanese)
print(X_japanese)

Search Engine:経済的
経済的


In [ ]:
data_chinese = LoadData().from_text("Japanes_Shinzo Abe Tweet 20171024 - Tweet.csv")
TextEmbedder().embed(corpus_list=data_chinese)
corpus_embedding = TextEmbedder().load_embedding()

Embedding data allready present, Do you want Embed & Save Again? Enter yes or no
yes



Embedding data Saved Successfully Again!
['corpus_list_data.pickle', 'corpus_embeddings_data.pickle']
Embedding data Loaded Successfully!
['corpus_list_data.pickle', 'corpus_embeddings_data.pickle']


In [ ]:
TextSearch().find_similar(query_text=X_japanese,top_n=10)

[{'index': 8,
  'score': 0.37281364,
  'text': 'https://twitter.com/AbeShinzo,安倍晋三,AbeShinzo,Oct 17,https://twitter.com/AbeShinzo/status/920292054663434245,私たち自民党は日本の経済を支えている中小企業を力強く応援したい。その思いから中小・小規模事業者の固定資産税を、３年間半減する制度を始め既に３万件近い利用がありました。この制度の実現により中小企業の倒産は3割も減少しました。更に生産性を高めるための支援を大胆に進めます。pic.twitter.com/N9oB57kACi,"We LDP would like to strongly support the small and medium-sized companies that are supporting the Japanese economy. The property tax of small and medium-sized and small businesses from the thought, there was already available 30,000 close, including a system to half three years. Bankruptcy of small and medium-sized companies by the implementation of this system has also been reduced by 30%. We boldly promoted the support of to further enhance the productivity. pic.twitter.com/N9oB57kACi",pic.twitter.com/N9oB57kACi,https://t.co/N9oB57kACi,529 replies,"5,336 retweets","11,611 likes",529,5.3K,12K'},
 {'index': 63,
  'score': 0.36950642,
  'text': 'https://twitter.com/AbeShinz